In [0]:
#L J Dataset
!wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2

--2019-12-15 02:13:48--  https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 174.138.79.61
Connecting to data.keithito.com (data.keithito.com)|174.138.79.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2.6G) [application/octet-stream]
Saving to: ‘LJSpeech-1.1.tar.bz2’

LJSpeech-1.1.tar.bz 100%[===================>]   2.56G  16.9MB/s    in 2m 38s  

2019-12-15 02:16:32 (16.6 MB/s) - ‘LJSpeech-1.1.tar.bz2’ saved [2748572632/2748572632]



In [0]:
!tar -xvf /content/LJSpeech-1.1.tar.bz2

In [0]:
!git clone https://github.com/ttaoREtw/Tacotron-pytorch.git

Cloning into 'Tacotron-pytorch'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 100 (delta 0), reused 0 (delta 0), pack-reused 97
Receiving objects: 100% (100/100), 67.77 MiB | 11.54 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [0]:
#Get Checkpoint 
!wget https://drive.google.com/uc?authuser=2&id=1q8xLo9zyyclIDgYk3V2mczofnQwqT6pk&export=download

--2019-12-15 01:24:29--  https://drive.google.com/uc?authuser=2
Resolving drive.google.com (drive.google.com)... 64.233.170.102, 64.233.170.139, 64.233.170.101, ...
Connecting to drive.google.com (drive.google.com)|64.233.170.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/uc [following]
--2019-12-15 01:24:29--  https://drive.google.com/uc
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 400 Bad Request
2019-12-15 01:24:29 ERROR 400: Bad Request.



In [0]:
!rm -r LJSpeech-1.1new

In [0]:
!mv  /content/LJSpeech-1.1 /content/Tacotron-pytorch/

In [0]:
cd Tacotron-pytorch/

/content/Tacotron-pytorch


In [0]:
cd /content/Tacotron-pytorch/LJSpeech-1.1/wavs

/content/Tacotron-pytorch/LJSpeech-1.1/wavs


In [0]:
str(1).zfill(2)

'01'

In [0]:
cd ..

/content


In [0]:
!unzip Archive.zip

Archive:  Archive.zip
  inflating: LJ001-0011.wav          
   creating: __MACOSX/
  inflating: __MACOSX/._LJ001-0011.wav  
  inflating: LJ001-0005.wav          
  inflating: __MACOSX/._LJ001-0005.wav  
  inflating: LJ001-0010.wav          
  inflating: __MACOSX/._LJ001-0010.wav  
  inflating: LJ001-0006.wav          
  inflating: __MACOSX/._LJ001-0006.wav  
  inflating: LJ001-0013.wav          
  inflating: __MACOSX/._LJ001-0013.wav  
  inflating: LJ001-0007.wav          
  inflating: __MACOSX/._LJ001-0007.wav  
  inflating: LJ001-0009.wav          
  inflating: __MACOSX/._LJ001-0009.wav  
  inflating: LJ001-0008.wav          
  inflating: __MACOSX/._LJ001-0008.wav  


In [0]:
for i in range(1,187):
  
  index=str(i).zfill(4)
  filename="LJ001-"+index+".wav"
  !cp -r  filename LJSpeech-1.1new

In [0]:
import os
count=0
for filename in os.listdir("/content/Tacotron-pytorch/LJSpeech-1.1/wavs"):
    # if filename.endswith(".wav"):
    #   if filename in listdir:
        print(filename)
        filename =str(filename)
        !cp -r  filename LJSpeech-1.1new

        break

LJ009-0188.wav
cp: cannot stat 'filename': No such file or directory


In [0]:
!cat wavs  

cat: wavs: Is a directory


In [0]:
#!find . -maxdepth 1 -type f |head -1000|xargs cp -t "LJSpeech-1.1new"

In [0]:
cd Tacotron-pytorch/

/content/Tacotron-pytorch


In [0]:
!pip install soundfile

In [0]:



def preprocess(args):
    with open(args.config) as f:
        config = yaml.load(f)

    # Make directory if not exist
    os.makedirs(args.output_dir, exist_ok=True)
    print('')
    print('[INFO] Root directory:', args.data_dir)

    AP = AudioProcessor(**config['audio'])
    executor = ProcessPoolExecutor(max_workers=args.n_jobs)
    fid = []
    text = []
    wav = []
    futures = []
    with open(args.old_meta, encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('|')
            fpath = os.path.join(args.data_dir, '%s.wav' % parts[0])
            text = parts[2]
            job = executor.submit(partial(process_utterance, fpath, text, args.output_dir, AP))
            futures += [job]

    print('[INFO] Preprocessing', end=' => ')
    print(len(futures), 'audio files found')
    results = [future.result() for future in tqdm(futures)]
    fpath_meta = os.path.join(args.output_dir, 'ljspeech_meta.txt')
    with open(fpath_meta, 'w') as f:
        for x in results:
            s = map(lambda x: str(x), x)
            f.write('|'.join(s) + '\n')


def process_utterance(fpath, text, output_dir,
        audio_processor, store_mel=True, store_linear=True):
    wav = audio_processor.load_wav(fpath)
    mel = audio_processor.melspectrogram(wav).astype(np.float32).T
    linear = audio_processor.spectrogram(wav).astype(np.float32).T
    n_frames = linear.shape[0]
    fid = fpath.split('/')[-1].split('.')[0]
    fpath_mel = fid + '-mel.npy'
    fpath_linear = fid + '-linear.npy'
    if store_mel:
        np.save(os.path.join(output_dir, fpath_mel), mel, allow_pickle=False)
    if store_linear:
        np.save(os.path.join(output_dir, fpath_linear), linear, allow_pickle=False)
    return fpath_mel, fpath_linear, n_frames, text




In [0]:
import os
import numpy as np
import argparse
import yaml
from tqdm import tqdm

import sys
# To import from src
sys.path.insert(0, '.')

from src.utils import AudioProcessor
from concurrent.futures import ProcessPoolExecutor
from functools import partial
from multiprocessing import cpu_count
from pathlib import Path

In [0]:
wav = AudioProcessor.load_wav(path="/content/wavs/LJ001-0005.wav")

TypeError: ignored

In [0]:
mel = audio_processor.melspectrogram(wav).astype(np.float32).T

In [0]:

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Preprocess ljspeech dataset')
    parser.add_argument('--data-dir', type=str, help='Directory to raw dataset')
    parser.add_argument('--output-dir', default='training_data/', type=str, help='Directory to store output', required=False)
    parser.add_argument('--old-meta', type=str, help='previous old meta file', required=True)
    parser.add_argument('--n-jobs', default=cpu_count(), type=int, help='Number of jobs used for feature extraction', required=False)
    parser.add_argument('--config', type=str, help='configure file', required=True)
    args = parser.parse_args()
    preprocess(args)

In [0]:
!python /content/Tacotron-pytorch/data/preprocess.py  --data-dir "content/wavs" --output-dir "content/training" --old-meta "/content/sample_metadata.csv" --config "config/config.yaml"


[INFO] Root directory: content/wavs
Traceback (most recent call last):
  File "/content/Tacotron-pytorch/data/preprocess.py", line 76, in <module>
    preprocess(args)
  File "/content/Tacotron-pytorch/data/preprocess.py", line 37, in preprocess
    text = parts[2]
IndexError: list index out of range


In [0]:
cd Tacotron-pytorch/

/content/Tacotron-pytorch


In [0]:
!pip install soundfile

In [0]:
!pip install tensorboardX

     |████████████████████████████████| 194kB 2.8MB/s 


In [0]:
!pip install unidecode

     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import librosa         # Process audio
import matplotlib      # Plot spectrogram and attention
import numpy           # Fundamental package for scientific computing
import scipy           # Use signal.lfilter
import yaml            # Config parser
import soundfile       # Save audio
import tensorboardX    # Tensorboard for pytorch
import tqdm            # Verbosity
import torch  

In [0]:
!python generate_speech.py --text "I was able to produce sound output using custom software" --checkpoint-path "/content/checkpoint_step138000 (1).pth"  --config config/config.yaml

Traceback (most recent call last):
  File "generate_speech.py", line 41, in <module>
    generate_speech(args)
  File "generate_speech.py", line 11, in generate_speech
    model = load_ckpt(config, args.checkpoint_path)
  File "generate_speech.py", line 24, in load_ckpt
    ckpt = torch.load(ckpt_path, map_location=torch.device('cpu'))
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 419, in load
    f = open(f, 'rb')
FileNotFoundError: [Errno 2] No such file or directory: '/content/checkpoint_step138000 (1).pth'


In [0]:
!pip install -r  used_packages.txt

ERROR: Could not find a version that satisfies the requirement yaml (from -r used_packages.txt (line 5)) (from versions: none)
ERROR: No matching distribution found for yaml (from -r used_packages.txt (line 5))


In [0]:
! pip install pyyaml

In [0]:
from concurrent.futures import ProcessPoolExecutor
from functools import partial

In [0]:
import numpy as np
import os
from utils import audio

ModuleNotFoundError: ignored